In [1]:
from fastai.vision import *
from pathlib import Path
import pickle
import sys
from operator import attrgetter
from PIL import Image
PATH = Path('data')
list(PATH.iterdir())

[PosixPath('data/challenge-2019-classes-description-500.csv'),
 PosixPath('data/fulltest'),
 PosixPath('data/challenge-2019-validation-detection-human-imagelabels.csv'),
 PosixPath('data/Xclassdescription.csv'),
 PosixPath('data/challenge-2019-validation-detection-bbox.csv'),
 PosixPath('data/fulltrain'),
 PosixPath('data/challenge-2019-train-detection-bbox.csv'),
 PosixPath('data/databunch_bs4_size256.pickle'),
 PosixPath('data/fullvalidation'),
 PosixPath('data/databunch_bs4_size128.pickle'),
 PosixPath('data/challenge-2019-train-detection-human-imagelabels.csv'),
 PosixPath('data/databunch_bs16_size256.pickle')]

In [2]:
class_names = pd.read_csv(PATH/'challenge-2019-classes-description-500.csv', header=None, names=["LabelName", "Class"])

In [3]:
# batch size
bs = 16
# image size
size = 256

#### Labels, bounding boxes for training

In [4]:
il_train = ImageList.from_folder(PATH/"fulltrain/", convert_mode='L') # change convert_mode to keep all color channels
bboxes_train = pd.read_csv(PATH/'challenge-2019-train-detection-bbox.csv')
labels_train = bboxes_train.merge(class_names, on="LabelName")
labels_train.loc[:, "ImageID"] = labels_train.loc[:, "ImageID"].astype('category')

In [ ]:
#%%script false
ids_train = list(map(attrgetter('stem'), list(il_train.items)))
df_dict = labels_train.to_dict("list")
#df_dict['X'], df_dict['Y'] = map(list, zip(*[Image.open(path).size for path in il_train.items])) #PIL-way
df_dict['Y'], df_dict['X'] = map(list, zip(*[tuple(open_image(path).shape[1:]) for path in il_train.items])) #fa-way
x = {image_id : {"bboxes" : [], "Classes" : []} for image_id in ids_train}
[x.update({image_id: {"bboxes" : x[image_id]["bboxes"] + [[xmin*X, xmax*X, ymin*Y, ymax*Y]],
                      "Classes" : x[image_id]["Classes"] + [c]}}) 
 for c, image_id, xmin, xmax, ymin, ymax, X, Y
 in zip(df_dict["Class"], df_dict["ImageID"], 
        df_dict["XMin"], df_dict["XMax"], 
        df_dict["YMin"], df_dict["YMax"], 
        df_dict['X'], df_dict['Y'])
]
img2bbox_train = {image_id : [x[image_id]["bboxes"], x[image_id]["Classes"]] for image_id in ids_train}

In [ ]:
%%script false
ids_train = list(map(attrgetter('stem'), list(il_train.items)))
df_dict = labels_train.to_dict("list")
x = {image_id : {"bboxes" : [], "Classes" : []} for image_id in ids_train}
[x.update({image_id: {"bboxes" : x[image_id]["bboxes"] + [[xmin*size, xmax*size, ymin*size, ymax*size]],
                      "Classes" : x[image_id]["Classes"] + [c]}}) 
 for c, image_id, xmin, xmax, ymin, ymax
 in zip(df_dict["Class"], df_dict["ImageID"], 
        df_dict["XMin"], df_dict["XMax"], 
        df_dict["YMin"], df_dict["YMax"])
]
img2bbox_train = {image_id : [x[image_id]["bboxes"], x[image_id]["Classes"]] for image_id in ids_train}

#### Labels, bounding boxes for validation

In [ ]:
il_val = ImageList.from_folder(PATH/"fullvalidation/", convert_mode='L') # change convert_mode to keep all color channels
bboxes_val = pd.read_csv(PATH/'challenge-2019-validation-detection-bbox.csv')
labels_val = bboxes_val.merge(class_names, on="LabelName")
labels_val.loc[:, "ImageID"] = labels_val.loc[:, "ImageID"].astype('category')

In [ ]:
#%%script false
ids_val = list(map(attrgetter('stem'), list(il_val.items)))
df_dict = labels_val.to_dict("list")
#df_dict['X'], df_dict['Y'] = map(list, zip(*[Image.open(path).size for path in il_train.items])) #PIL-way
df_dict['Y'], df_dict['X'] = map(list, zip(*[tuple(open_image(path).shape[1:]) for path in il_val.items])) #fa-way
x = {image_id : {"bboxes" : [], "Classes" : []} for image_id in ids_val}
[x.update({image_id: {"bboxes" : x[image_id]["bboxes"] + [[xmin*X, xmax*X, ymin*Y, ymax*Y]],
                      "Classes" : x[image_id]["Classes"] + [c]}}) 
 for c, image_id, xmin, xmax, ymin, ymax, X, Y
 in zip(df_dict["Class"], df_dict["ImageID"], 
        df_dict["XMin"], df_dict["XMax"], 
        df_dict["YMin"], df_dict["YMax"], 
        df_dict['X'], df_dict['Y'])
]
img2bbox_val = {image_id : [x[image_id]["bboxes"], x[image_id]["Classes"]] for image_id in ids_val}

In [ ]:
%%script false
ids_val = list(map(attrgetter('stem'), list(il_val.items)))
df_dict = labels_val.to_dict("list")
x = {image_id : {"bboxes" : [], "Classes" : []} for image_id in ids_val}
[x.update({image_id: {"bboxes" : x[image_id]["bboxes"] + [[xmin*size, xmax*size, ymin*size, ymax*size]],
                      "Classes" : x[image_id]["Classes"] + [c]}}) 
 for c, image_id, xmin, xmax, ymin, ymax
 in zip(df_dict["Class"], df_dict["ImageID"], 
        df_dict["XMin"], df_dict["XMax"], 
        df_dict["YMin"], df_dict["YMax"])
]
img2bbox_val = {image_id : [x[image_id]["bboxes"], x[image_id]["Classes"]] for image_id in ids_val}

In [ ]:
img2bbox = {**img2bbox_train, **img2bbox_val}

In [ ]:
get_y_func = lambda o:img2bbox[o.stem]

In [ ]:
data = (ObjectItemList
       .from_folder(path=PATH, convert_mode='L')
       .split_by_folder(train='fulltrain', valid='fullvalidation')
       .label_from_func(get_y_func)
       .transform(get_transforms(), size=size, tfm_y=True)
       .databunch(path=PATH, bs=bs, collate_fn=bb_pad_collate))

In [ ]:
with open(PATH/'databunch_bs{}_size{}.pickle'.format(bs, size), 'wb') as f:
    pickle.dump(data, f)

In [ ]:
data.show_batch(rows=4)

In [ ]:
# Notes:
# Somehow, images have only 1 channel (set convert_mode to something other than L to change that!!)
# PIL library does not correctly read some image metadata therefore 

In [ ]:
d = dict()
d['X'],d['Y'] = map(list, zip(*[Image.open(path).size for path in il_train.items[:10]]))
d

In [ ]:
l = il_train.items[:10]

In [ ]:
# the PIL-way:
%timeit df_dict['X'], df_dict['Y'] = map(list, zip(*[Image.open(path).size for path in l]))

In [ ]:
# the fastai-way:
%timeit df_dict['Y'], df_dict['X'] = map(list, zip(*[tuple(open_image(path).shape[1:]) for path in l]))

Continuing like in Pascal NB

In [ ]:
%%script false 
#Grab the convenience functions that helps us build the Unet
from fastai.vision.models.unet import _get_sfs_idxs, model_sizes, hook_outputs

class LateralUpsampleMerge(nn.Module):
    "Merge the features coming from the downsample path (in `hook`) with the upsample path."
    def __init__(self, ch, ch_lat, hook):
        super().__init__()
        self.hook = hook
        self.conv_lat = conv2d(ch_lat, ch, ks=1, bias=True)
    
    def forward(self, x):
        return self.conv_lat(self.hook.stored) + F.interpolate(x, self.hook.stored.shape[-2:], mode='nearest')

class RetinaNet(nn.Module):
    "Implements RetinaNet from https://arxiv.org/abs/1708.02002"
    def __init__(self, encoder:nn.Module, n_classes, final_bias=0., chs=256, n_anchors=9, flatten=True):
        super().__init__()
        self.n_classes,self.flatten = n_classes,flatten
        imsize = (256,256)
        sfs_szs = model_sizes(encoder, size=imsize)
        sfs_idxs = list(reversed(_get_sfs_idxs(sfs_szs)))
        self.sfs = hook_outputs([encoder[i] for i in sfs_idxs])
        self.encoder = encoder
        self.c5top5 = conv2d(sfs_szs[-1][1], chs, ks=1, bias=True)
        self.c5top6 = conv2d(sfs_szs[-1][1], chs, stride=2, bias=True)
        self.p6top7 = nn.Sequential(nn.ReLU(), conv2d(chs, chs, stride=2, bias=True))
        self.merges = nn.ModuleList([LateralUpsampleMerge(chs, sfs_szs[idx][1], hook) 
                                     for idx,hook in zip(sfs_idxs[-2:-4:-1], self.sfs[-2:-4:-1])])
        self.smoothers = nn.ModuleList([conv2d(chs, chs, 3, bias=True) for _ in range(3)])
        self.classifier = self._head_subnet(n_classes, n_anchors, final_bias, chs=chs)
        self.box_regressor = self._head_subnet(4, n_anchors, 0., chs=chs)
        
    def _head_subnet(self, n_classes, n_anchors, final_bias=0., n_conv=4, chs=256):
        "Helper function to create one of the subnet for regression/classification."
        layers = [conv_layer(chs, chs, bias=True, norm_type=None) for _ in range(n_conv)]
        layers += [conv2d(chs, n_classes * n_anchors, bias=True)]
        layers[-1].bias.data.zero_().add_(final_bias)
        layers[-1].weight.data.fill_(0)
        return nn.Sequential(*layers)
    
    def _apply_transpose(self, func, p_states, n_classes):
        #Final result of the classifier/regressor is bs * (k * n_anchors) * h * w
        #We make it bs * h * w * n_anchors * k then flatten in bs * -1 * k so we can contenate
        #all the results in bs * anchors * k (the non flatten version is there for debugging only)
        if not self.flatten: 
            sizes = [[p.size(0), p.size(2), p.size(3)] for p in p_states]
            return [func(p).permute(0,2,3,1).view(*sz,-1,n_classes) for p,sz in zip(p_states,sizes)]
        else:
            return torch.cat([func(p).permute(0,2,3,1).contiguous().view(p.size(0),-1,n_classes) for p in p_states],1)
    
    def forward(self, x):
        c5 = self.encoder(x)
        p_states = [self.c5top5(c5.clone()), self.c5top6(c5)]
        p_states.append(self.p6top7(p_states[-1]))
        for merge in self.merges: p_states = [merge(p_states[0])] + p_states
        for i, smooth in enumerate(self.smoothers[:3]):
            p_states[i] = smooth(p_states[i])
        return [self._apply_transpose(self.classifier, p_states, self.n_classes), 
                self._apply_transpose(self.box_regressor, p_states, 4),
                [[p.size(2), p.size(3)] for p in p_states]]
    
    def __del__(self):
        if hasattr(self, "sfs"): self.sfs.remove()